In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial
import time
from picoscope import ps5000a
import picosdk
from picosdk.discover import find_all_units
import serial.tools.list_ports as port_list
import chipwhisperer as cw

## Program target with Design_Start soft core and AES firmware ##

In [2]:
#bitstream = r"/home/sca.user/Desktop/src/vivado/CW305_DesignStart.bit"
bitstream = r"//home/sca.user/Desktop/src/vivado/CW305_DesignStart/CW305_DesignStart.runs/impl_1/CW305_designstart_top.bit"

ftarget = cw.target(None, cw.targets.CW305, bsfile=bitstream, force=False)
print(ftarget.is_programmed())
print(ftarget.get_fpga_buildtime())

True
addr 11
0/0/2000, 00:00


##  Setup picoscope ##

In [ ]:
scopes = find_all_units()
for scope in scopes:
    print("Working with:")
    print(scope.info)
    scope.close()
ports = list(port_list.comports())
for p in ports:
    print (p)

ps = ps5000a.PS5000a()

# Since target runnning at 10 MHz then AES requires from trigger 3.225 uS
obs_duration = 3.225E-6
# Sample at least 1260 points within that window
sampling_interval = obs_duration / 1260
# Configure timebase
(actualSamplingInterval, nSamples, maxSamples) = ps.setSamplingInterval(sampling_interval, obs_duration)
print("Nsamples : ", nSamples)
print("Sampling interval = %f us" % (actualSamplingInterval*nSamples*1E6))

# 50mV range on channel A, AC coupled, 20 MHz BW limit
ps.setChannel('A', 'AC', 0.05, 0.0, enabled=True, BWLimited=True)
# Channel B is trigger
ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True)
ps.setSimpleTrigger('B', 2.0, 'Rising', timeout_ms=2000, enabled=True)

## Setup target platform ##

In [ ]:
# set clock:
ftarget.vccint_set(1.0)
# we only need PLL1:
ftarget.pll.pll_enable_set(True)            # enable PLL chip
ftarget.pll.pll_outenable_set(False, 0)     # disable PLL 0
ftarget.pll.pll_outenable_set(True, 1)      # enable PLL 1 
ftarget.pll.pll_outenable_set(False, 2)     # disable PLL 2

# run at 10 MHz:
ftarget.pll.pll_outfreq_set(10E6, 1)

## Run capture  ##

Convenience functions

In [ ]:
# capture functions
def pico_capture():
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    ftarget.fpga_write(ftarget.REG_USER_LED, [0x01])
    ftarget.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data

# some convenience functions:
def reset_fpga():
    # resets the full CW305 FPGA
    ftarget.fpga_write(3, [1])
    ftarget.fpga_write(3, [0])

def reset_arm_target(ftarget):
    # resets only the Arm DesignStart core within the CW305 FPGA
    ftarget.fpga_write(2, [1])
    ftarget.fpga_write(2, [0])
    
def reset_flush():
    reset_arm_target(ftarget)
    ftarget.flush()

Capture loop

In [ ]:
from Crypto.Cipher import AES
from chipwhisperer.common.traces import Trace

reset_flush()
ktp = cw.ktp.Basic()
# Initialize cipher to verify DUT result:
key, plaintext = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)
print("Key: ", [hex(el) for el in key])
print("Plaintext: ", [hex(el) for el in plaintext])

output_len = 16
# Dummy capture call due to bug of using AC coupling
pico_capture()

# endieness of the message already inverted by function 
ftarget.simpleserial_write('k', key)
key_written = ftarget.fpga_read(ftarget.REG_CRYPT_KEY, output_len)
print("\nValue in REG_CRYPT_KEY: ", [hex(el) for el in key_written])
ftarget.simpleserial_write('p', plaintext)
text_written = ftarget.fpga_read(ftarget.REG_CRYPT_TEXTIN, output_len)
print("Value in REG_CRYPT_TEXTIN: ", [hex(el) for el in text_written])
response = ftarget.simpleserial_read('r', output_len, ack=True)
print("Initial value in REG_CRYPT_CIPHEROUT: ", [hex(el) for el in response])
wave = pico_capture()

response = ftarget.simpleserial_read('r', output_len, ack=True)
print("\nCiphertext: ", [hex(el) for el in response])
print("Expected : ", [hex(el) for el in cipher.encrypt(bytes(plaintext))])
print("REG_CRYPT_CIPHEROUT: ", [hex(el) for el in response])
print("REG_CRYPT_TEXTIN: ", [hex(el) for el in text_written])
print("REG_CRYPT_KEY: ", [hex(el) for el in key_written])

# Disconnect for all devices
ftarget.dis()

assert (list(response) == list(cipher.encrypt(bytes(plaintext)))), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), list(plaintext))
    

Plotting trace

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure(plot_width=800)

xrange = range(len(wave))
p.line(xrange, wave, line_color="red")
show(p)